In [ ]:
import numpy as np
import torch
import torchvision
from torchvision import datasets
from torchvision import transforms
from tqdm import tqdm
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#Ссылка на датасет https://www.kaggle.com/datasets/moazeldsokyx/dogs-vs-cats
#Использую test и validation вборки, потому что train очень большая и не помещается в оперативную память

In [ ]:
# Создание и нормализация обучающей, тестовой и выборки валидации

batch_size = 32

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize((256, 256)),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_path = "C:/Users/das-s/Downloads/DogsCats/dataset/test"
train_ds = datasets.ImageFolder(root=train_path, transform=transform)
train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

test_path = "C:/Users/das-s/Downloads/DogsCats/dataset/validation"
test_ds = datasets.ImageFolder(root=test_path, transform=transform)
test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
images = []
for i in range(4):
    images.append(train_features[i])
imshow(torchvision.utils.make_grid(images)) # show images
print(' '.join(f'{train_ds.classes[train_labels[j]]}\t' for j in range(4))) # print labels


In [ ]:
# Функция формирования набора данных (картинки и метки)
def extract_dataset_images(dataloader):
  labels = []
  images = []
  for batch, batch_labels in tqdm(dataloader):
    images.append(batch.detach().numpy().reshape((batch.shape[0], -1))) # Делаем картинки в виде векторов
    labels.append(batch_labels.detach().numpy())
  return np.concatenate(images), np.concatenate(labels)

In [ ]:
x_train, y_train = extract_dataset_images(train_dataloader)
x_test, y_test = extract_dataset_images(test_dataloader)

# KNN

In [ ]:
def train_and_test_KNN(x_train, y_train, x_test, y_test, n_neighbors, metric, weights):
  # Обучение KNN на картинках
  knn_classifier = KNeighborsClassifier(n_neighbors=n_neighbors, metric = metric, weights = weights)
  knn_classifier.fit(x_train, y_train)

  # Тестирование
  # Классификация тестовых изображений с использованием KNN
  y_pred = knn_classifier.predict(x_test)

  accuracy = accuracy_score(y_test, y_pred)
  return accuracy

In [ ]:
def inverse_squared_distance(dist):
    arr = []
    for d in dist:
        arr.append(1 / (d**2))
    return arr

In [ ]:
metrics_array = ["minkowski", "euclidean", "cosine"]
weights_array = ["uniform", "distance", inverse_squared_distance]
n_neighbors = [3, 5, 10, 20, 30]

In [ ]:
best_accuracy = 0
best_n_neighbors = 0
best_metric = ""
best_weights = ""
for metric in metrics_array:
    for weights in weights_array:
        for n in n_neighbors:
            accuracy = train_and_test_KNN(x_train, y_train, x_test, y_test, n, metric, weights)
            print(f"n = {n}, metric = {metric}, weights = {weights} \naccuracy = {accuracy:.3f}\n")
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_n_neighbors = n
                best_metric = metric
                best_weights = weights
print('-'*50)
print("Best KNN accuracy:")
print(f"n_neighbors = {best_n_neighbors} \nmetric = {best_metric} \nweights = {best_weights} \naccuracy = {best_accuracy}")

In [ ]:
knn_classifier = KNeighborsClassifier(n_neighbors=best_n_neighbors, metric=best_metric, weights=best_weights)
knn_classifier.fit(x_train, y_train)
y_pred = knn_classifier.predict(x_test)
print(classification_report(y_test, y_pred, target_names=test_ds.classes))

# PCA+KNN

In [ ]:
def train_and_test_PCA_KNN(x_train, y_train, x_test, y_test, n_components, knn_classifier):
  pca = PCA(n_components=n_components)
  x_train_pca = pca.fit_transform(x_train)
  knn_classifier.fit(x_train_pca, y_train)

  # Тестирование
  # Применение созданного ранее PCA для тестовых данных
  x_test_pca = pca.transform(x_test)
  # Классификация тестовых изображений с использованием KNN
  y_pred = knn_classifier.predict(x_test_pca)
  accuracy = accuracy_score(y_test, y_pred)
  return accuracy

In [ ]:
n_components = [10, 50, 100, 200, 300, 500, 700]

In [ ]:
best_accuracy = 0
best_n_neighbors = 0
best_metric = ""
best_weights = ""
best_n_components = 0

for metric in metrics_array:
    for weights in weights_array:
        for n_n in n_neighbors:
            for n_c in n_components:
                knn_classifier = KNeighborsClassifier(n_neighbors=n_n, metric = metric, weights = weights)
                accuracy = train_and_test_PCA_KNN(x_train, y_train, x_test, y_test, n_c, knn_classifier)
                print(f"n_neighbors = {n_n}, metric = {metric}, weights = {weights}, n_components = {n_c}")
                print(f"accuracy = {accuracy:.3f}\n")
                print('-'*50)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_n_neighbors = n_n
                    best_metric = metric
                    best_weights = weights
                    best_n_components = n_c
print('-'*50)
print("Best PCA+KNN accuracy:")
print(f"n_neighbors = {best_n_neighbors} \nmetric = {best_metric} \nweights = {best_weights}")
print(f"n_components = {best_n_components} \naccuracy = {best_accuracy:.3f}")

In [ ]:
knn_classifier = KNeighborsClassifier(n_neighbors=best_n_neighbors, metric=best_metric, weights=best_weights)
pca = PCA(n_components=best_n_components)

x_train_pca = pca.fit_transform(x_train)
knn_classifier.fit(x_train_pca, y_train)

x_test_pca = pca.transform(x_test)
y_pred = knn_classifier.predict(x_test_pca)

print(classification_report(y_test, y_pred, target_names=test_ds.classes))

# CNN+PCA+KNN

In [ ]:
from torchvision.models import googlenet
model = googlenet(pretrained=True)
model.classifier = nn.Flatten(start_dim= 1)
model.to(device)

In [ ]:
def extract_features(dataset, model):
    features = []
    labels = []
    for batch_images, batch_labels in tqdm(dataset):
        batch_features = model(batch_images.to(device))
        labels.append(batch_labels.detach().numpy())
        features.append(batch_features.cpu().detach().numpy())
    return np.concatenate(features), np.concatenate(labels)

In [ ]:
x_train_features, y_train_labels = extract_features(train_dataloader, model)
x_test_features, y_test_labels = extract_features(test_dataloader, model)

In [ ]:
best_accuracy = 0
best_n_neighbors = 0
best_metric = ""
best_weights = ""
best_n_components = 0

for metric in metrics_array:
    for weights in weights_array:
        for n_n in n_neighbors:
            for n_c in n_components:
                knn_classifier = KNeighborsClassifier(n_neighbors=n_n, metric = metric, weights = weights)
                accuracy = train_and_test_PCA_KNN(x_train_features, y_train_labels, x_test_features, y_test_labels, n_c, knn_classifier)
                print(f"n_neighbors = {n_n}, metric = {metric}, weights = {weights}, n_components = {n_c}")
                print(f"accuracy = {accuracy:.3f}\n")
                print('-'*50)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_n_neighbors = n_n
                    best_metric = metric
                    best_weights = weights
                    best_n_components = n_c
print('-'*50)
print("Best PCA+KNN+CNN accuracy:")
print(f"n_neighbors = {best_n_neighbors} \nmetric = {best_metric} \nweights = {best_weights}")
print(f"n_components = {best_n_components} \naccuracy = {best_accuracy:.3f}")

In [ ]:
knn_classifier = KNeighborsClassifier(n_neighbors=best_n_neighbors, metric=best_metric, weights=best_weights)
pca = PCA(n_components=best_n_components)

x_train_pca = pca.fit_transform(x_train_features)
knn_classifier.fit(x_train_pca, y_train_labels)

x_test_pca = pca.transform(x_test_features)
y_pred = knn_classifier.predict(x_test_pca)

print(classification_report(y_test_labels, y_pred, target_names=test_ds.classes))